# Figures

A notebook to make figures of mesh differences

In [90]:
import json
import pyvista as pv
import pymesh
from helpers.mesh import *
import cityjson
import pandas as pd
import os

def rpath(path):
    return os.path.expanduser(path)

df = pd.read_csv(rpath("~/3DBAG_09/all/lod2.2.csv"))

In [91]:
def load_citymodel(file):
    cm = json.load(file)

    if "transform" in cm:
        s = cm["transform"]["scale"]
        t = cm["transform"]["translate"]
        verts = [[v[0] * s[0] + t[0], v[1] * s[1] + t[1], v[2] * s[2] + t[2]]
                for v in cm["vertices"]]
    else:
        verts = cm["vertices"]

    # mesh points
    vertices = np.array(verts)

    return cm, vertices

def get_geometry(co, lod):
    """Returns the geometry of the given LoD.
    
    If lod is None then it returns the first one.
    """

    if len(co["geometry"]) == 0:
        return None

    if lod is None:
        return co["geometry"][0]

    for geom in co["geometry"]:
        if str(geom["lod"]) == str(lod):
            return geom

def is_valid(mesh):
    return mesh.volume > 0 and mesh.n_open_edges == 0

def load_building(objid, tile_id=None, tile_csv=None, lod="2.2"):    
    if tile_id is None:
        tile_id = tile_csv.set_index("id").loc[objid]["tile_id"]

    filename = rpath(f"~/3DBAG_09/{tile_id}.json")

    with open(filename, "rb") as file:
        cm, verts = load_citymodel(file)

    building = cm["CityObjects"][objid]

    geom = get_geometry(building, lod)
    mesh = cityjson.to_triangulated_polydata(geom, verts)
    
    return mesh, geom, verts

In [ ]:
selected_ids = [
    "NL.IMBAG.Pand.0599100000702379-0",
    "NL.IMBAG.Pand.0518100001635181-0",
    "NL.IMBAG.Pand.0599100000701103-0",
    "NL.IMBAG.Pand.0518100000225439-0",
    "NL.IMBAG.Pand.0518100000273015-0",
    "NL.IMBAG.Pand.0363100012075730-0",
    "NL.IMBAG.Pand.0363100012185598-0",
    "NL.IMBAG.Pand.0344100000031226-0",
    "NL.IMBAG.Pand.0344100000077683-0",
    "NL.IMBAG.Pand.0344100000099499-0",
    "NL.IMBAG.Pand.0599100000080428-0",
    "NL.IMBAG.Pand.0518100000230634-0",
    "NL.IMBAG.Pand.0518100000206625-0",
    "NL.IMBAG.Pand.0518100000226316-0",
    "NL.IMBAG.Pand.0518100000282020-0",
    "NL.IMBAG.Pand.0518100000222277-0",
    "NL.IMBAG.Pand.0629100000020777-0",
    "NL.IMBAG.Pand.0363100012236081-0",
    "NL.IMBAG.Pand.0599100000432858-0",
    "NL.IMBAG.Pand.0518100000206625-0"
]

footprints = []
for objid in selected_ids:
    mesh, geom, verts = load_building(objid, tile_csv=df)
    
    footprint = cityjson.to_shapely(geom, verts, ground_only=True)
    
    footprints.append([objid, footprint])

In [ ]:
import geopandas

footprints = geopandas.GeoDataFrame(footprints, columns=["id", "geometry"], geometry="geometry")

In [ ]:
footprints.to_file("footprints.gpkg", driver="GPKG")

# Single building plots

First, let's load the building (we can load two different LoDs):

In [ ]:
# Duplicate to save time from loading every time

# For figure 1
# NL.IMBAG.Pand.0603100000009302-0
# NL.IMBAG.Pand.0489100000210413-0
# NL.IMBAG.Pand.0344100000045583-0

# objid = "NL.IMBAG.Pand.0489100000210413-0"

# Example building
#  NL.IMBAG.Pand.0518100000337631

# Building with one hole
# NL.IMBAG.Pand.0599100000601466-0
objid="NL.IMBAG.Pand.0518100000337631-0"

def load_bagid(objid, lod="2.2"):
    
    tile_id = df.set_index("id").loc[objid]["tile_id"]

# NL.IMBAG.Pand.0599100000763318 - Van Nelle

    filename = rpath(f"~/3DBAG_09/{tile_id}.json")

    with open(filename, "rb") as file:
        cm, verts = load_citymodel(file)

    building = cm["CityObjects"][objid]
    
    geom = get_geometry(building, lod)
    
    mesh = cityjson.to_triangulated_polydata(geom, verts)
    
    return mesh, geom, verts

lod_source = "1.2"
lod_dest = "2.2"

mesh_source, geom_source, verts = load_bagid(objid, lod_source)
mesh_dest, geom_dest, verts = load_bagid(objid, lod_dest)

In [95]:
p = pv.Plotter(window_size=[2048, 2048])

p.background_color = "white"

# p.enable_parallel_projection()

p.add_mesh(mesh_dest.extract_feature_edges(), color="black", line_width=5)
p.add_mesh(mesh_dest, scalars="semantics", cmap=["black", "red", "lightgrey"], color="lightgrey", ambient=0.7, diffuse=0.5)

p.show()

p.save_graphic(rpath("~/figures/simple/example-main-semantics.pdf"))

ViewInteractiveWidget(height=2048, layout=Layout(height='auto', width='100%'), width=2048)

## Plot boolean operations

This will plot the boolean operation outcome of the loaded LoDs:

In [ ]:
pm_source = to_pymesh(mesh_source)
pm_dest = to_pymesh(mesh_dest)

inter = intersect(pm_source, pm_dest)
diff = difference(pm_source, pm_dest)
op_diff = difference(pm_dest, pm_source)
sym_diff = symmetric_difference(pm_source, pm_dest)

inter_vista = to_pyvista(inter)
inter_vista["source"] = inter.get_attribute("source")

diff_vista = to_pyvista(diff)
diff_vista["source"] = diff.get_attribute("source")

op_diff_vista = to_pyvista(op_diff)
op_diff_vista["source"] = op_diff.get_attribute("source")

sym_diff_vista = to_pyvista(sym_diff)
sym_diff_vista["source"] = sym_diff.get_attribute("source")

p = pv.Plotter(window_size=[4096, 640], shape=(1,7))

p.background_color = "white"

p.add_mesh(mesh_source, color="blue", opacity=0.5)
p.add_mesh(mesh_dest, color="orange", opacity=0.5)

p.add_mesh(mesh_source.extract_feature_edges(), color="blue", line_width=3, label=lod_source)
p.add_mesh(mesh_dest.extract_feature_edges(), color="orange", line_width=3, label=lod_dest)

p.subplot(0,1)

p.add_mesh(mesh_source, color="lightgrey")

p.subplot(0,2)

p.add_mesh(mesh_dest, color="lightgrey")

p.subplot(0,3)

# p.add_mesh(inter_vista, scalars="source", cmap=["orange", "blue"], label="Intersection")

p.add_mesh(inter_vista, color="lightgrey", label="Intersection")

p.subplot(0,4)

p.add_mesh(sym_diff_vista, color="lightgrey", label="Intersection")

p.subplot(0,5)

p.add_mesh(diff_vista, color="lightgrey", label="Intersection")

p.subplot(0,6)

p.add_mesh(op_diff_vista, color="lightgrey", label="Intersection")

p.show()
# p.save_graphic(f"{objid}-boolean.pdf", raster=False)

## Plot building and its related volumes (convex hull, OOBB and AABB)

This will plot a figure with four subfigures showing the building and its convex hull, OOBB and AABB:

In [ ]:
# --- End of duplicate

import pymesh
import cityjson
import geometry

# solid_color = "lightgrey"
# trans_color = "red"
# opacity = 0.7
# edges_on = True
# border_width = 2.0

# rot_angle = 0

# from shapely.geometry import Point
# obb_2d = Point(0, 0).buffer(10).minimum_rotated_rectangle
# mesh = pv.Sphere(10).clip("z", invert=False)

# A palette of colours (actually materials) to use
colour_palette = {
    "grey": {
        "mesh_color": "grey",
        "ambient": 0.9,
        "diffuse": 0.3
    },
    "pastel_green": {
        "mesh_color": "#b6e2d3",
        "ambient": 0.6,
        "diffuse": 0.1
    },
    "pastel_blue": {
        "mesh_color": "#81abbc",
        "ambient": 0.6,
        "diffuse": 0.1
    },
    "pastel_red": {
        "mesh_color": "#fbd2c9",
        "ambient": 0.6,
        "diffuse": 0.1
    },
    "pastel_yellow": {
        "mesh_color": "#fff4bd",
        "ambient": 0.6,
        "diffuse": 0.1
    }
}

def plot(mesh,
         geom,
         verts,
         mesh_color="lightgrey",
         ambient=0.0,
         diffuse=1.0,
         trans_color="red",
         opacity=0.7,
         edges_on=True,
         solid_edges="black",
         border_width=5.0,
         rot_angle=0,
         label=None,
         filename=None):
    """Plots the mesh provided alongside its convex hull, OOBB and AABB."""

    pm_mesh = to_pymesh(mesh)

    pm_ch = pymesh.convex_hull(pm_mesh)

    convex_hull = to_pyvista(pm_ch)

    # Compute OBB with shapely
    obb_2d = cityjson.to_shapely(geom, verts, ground_only=False).minimum_rotated_rectangle
    min_z = np.min(mesh.clean().points[:, 2])
    max_z = np.max(mesh.clean().points[:, 2])
    obb = geometry.extrude(obb_2d, min_z, max_z)

    points = np.array([[p[0], p[1], min_z] for p in obb_2d.boundary.coords])
    obb.points += np.mean(points, axis=0)

    aobb = mesh.outline(generate_faces=True)

    p = pv.Plotter(window_size=[2048, 2048], shape=(2,2))
    # p.add_title("test", color="black")

    p.background_color = "white"

    centroid = [(mesh.bounds[0] + mesh.bounds[1]) / 2, (mesh.bounds[2] + mesh.bounds[3]) / 2, (mesh.bounds[4] + mesh.bounds[5]) / 2]
    mesh.rotate_z(rot_angle, centroid)

    p.add_mesh(mesh, color=mesh_color, ambient=ambient, diffuse=diffuse)
    if edges_on:
        p.add_mesh(mesh.extract_feature_edges(feature_angle=10), color=solid_edges, line_width=border_width)

    p.subplot(0, 1)

    convex_hull.rotate_z(rot_angle, centroid)

    p.add_mesh(mesh, color=mesh_color, ambient=ambient, diffuse=diffuse)
    p.add_mesh(convex_hull, color=trans_color, opacity=opacity)
    if edges_on:
        p.add_mesh(convex_hull.extract_feature_edges(feature_angle=10), color=trans_color, line_width=border_width)

    p.subplot(1, 0)

    obb.rotate_z(rot_angle, centroid)

    p.add_mesh(mesh, color=mesh_color, ambient=ambient, diffuse=diffuse)
    p.add_mesh(obb, color=trans_color, opacity=opacity)
    if edges_on:
        p.add_mesh(obb.extract_feature_edges(), color=trans_color, line_width=border_width)

    p.subplot(1, 1)

    aobb.rotate_z(rot_angle, centroid)

    p.add_mesh(mesh, color=mesh_color, ambient=ambient, diffuse=diffuse)
    p.add_mesh(aobb, color=trans_color, opacity=opacity)
    if edges_on:
        p.add_mesh(aobb.extract_feature_edges(), color=trans_color, line_width=border_width)

    p.show()
    p.reset_camera()
    p.link_views()
    
    if not filename is None:
        p.save_graphic(filename)
    
    p.close()
# p.set_position(np.mean(mesh.points, axis=0) + [50, 0, 30])
# objid="hemisphere"

plot(mesh_dest, geom_dest, verts, border_width=2.0, **colour_palette["grey"], solid_edges="white", filename=rpath("~/figures/example-volumes.pdf"))

## Plot mesh and its voxels

In [ ]:
voxel = pv.voxelize(mesh_dest, density=0.5, check_surface=False)
# voxel.plot(show_edges=True, text=f"[{objid}] Voxelized")

p = pv.Plotter(window_size=[2048, 2048])

p.background_color = "white"

p.add_mesh(voxel, line_width=2.0, color='lightgrey', ambient=0.5, diffuse=0.8, show_edges=True)
# p.add_mesh(voxel.extract_all_edges(), line_width=2.0, color='white')
# p.add_mesh(voxel.cell_centers(), color='black')
# p.add_mesh(mesh_dest, color="grey")
# p.add_mesh(mesh_dest.extract_feature_edges(feature_angle=10), color='black')
# p.add_mesh(pv.PolyData(np.mean(voxel.cell_centers().points, axis=0)), color='white')

p.show()

p.save_graphic(rpath("~/figures/example-grid.pdf"))

## Plot mesh with surface grid

This will create a surface grid for the mesh and plot it on top of it:

In [ ]:
from shape_index import create_surface_grid

sgrid = pv.PolyData(create_surface_grid(mesh_dest, 0.6))

p = pv.Plotter(window_size=[2048, 2048])

p.add_mesh(mesh_dest, color="lightgrey", ambient=1, diffuse=0)
p.add_mesh(mesh_dest.extract_feature_edges(), color="black", line_width=10)
p.add_mesh(sgrid, render_points_as_spheres=True, point_size=10, color="black")

p.show()

In [ ]:
p.save_graphic(rpath("~/figures/example-surface-grid.pdf"))

## Plot the mesh with holes (if any)

First, let's find a mesh with holes:

In [ ]:
objid = "NL.IMBAG.Pand.0599100000254048-0"
mesh_holes, _, _ = load_bagid(objid, "2.2")

In [ ]:
edges = mesh_holes.extract_feature_edges(boundary_edges=True,
                           feature_edges=False,
                           manifold_edges=False)

p = pv.Plotter(window_size=[2048, 1024])

p.add_mesh(mesh_holes, color="grey", ambient=0.9, diffuse=0.3)
p.add_mesh(mesh_holes.extract_feature_edges(boundary_edges=False), color="black", line_width=3)
if mesh_holes.n_open_edges:
    p.add_mesh(edges, color='red', line_width=6)

# p.add_title(f"{objid} {'is watertight' if mesh_holes.n_open_edges == 0 else f'has {mesh_holes.n_open_edges} open edges'}", 8)    

# Zoom in to the hole of NL.IMBAG.Pand.0599100000254048-0
# p.camera.position = (93492.6, 439805, 79.0279)
# p.camera.focal_point = (93446.4, 439759, 32.7941)

p.show()
p.save_graphic(rpath("~/figures/example-holes.pdf"))
p.close()

# Figures about clustering

A set of figures related to some cluster analysis conducted in the Analysis notebook.

Let's load the data:

In [ ]:
clustering = pd.read_csv("clustering_200k_30n_11f_average.csv")

# show_n = 5

# p = pv.Plotter(shape=(1, show_n))

# sample = clustering[clustering["cluster"] != 5].sample(n = show_n)

# selected_ids = sample["id"]

# i = 0
# for objid in selected_ids:
#     p.subplot(0, i)

#     mesh, geom, verts = load_building(objid, tile_csv=df)
#     p.add_title(str(sample.iloc[i]["cluster"]))
#     p.add_mesh(mesh)
    
#     i += 1

# p.show()

Let's see the distribution of buildings among clusters:

In [ ]:
cluster_stats = pd.DataFrame(clustering.groupby("cluster").size(), columns=["count"])
cluster_stats["perc"] = cluster_stats["count"] / len(clustering)
cluster_stats.sort_values(["perc"], ascending=False)

In [ ]:
from plotnine import *
%matplotlib inline

(ggplot(clustering)         # defining what data to use
 + aes(x="cluster")    # defining what variable to use
 + geom_bar()  # defining the type of plot to use
 + scale_y_continuous(trans="log10")
 + coord_flip()
)

## Clusters plots of samples

This will plot samples of 9 (or less) buildings per cluster:

In [ ]:
import math

def get_shape(c):
    """Returns the most orthogonal shape that can reach a certain size"""
    high = math.floor(math.sqrt(c))
    
    return high, math.floor(c / high)

def plot_cluster(cluster_label,
                 show_class=True,
                 shape=(3,3),
                 **kwargs
                ):
    """Plots the specified cluster. If `cell_size` is set, then the `window_size` is ignored."""
    
    size_x = shape[0]
    size_y = shape[1]

    show_n = size_x * size_y

    label_df = clustering[clustering["cluster"] == cluster_label]
    
    if len(label_df) > show_n:
        sample = label_df.sample(n = show_n)
    else:
        sample = label_df
    
    return plot_buildings(sample, label=str(cluster_label) if show_class else None, shape=shape, **kwargs)
    
def plot_buildings(sample,
                 shape=(3,3),
                 allow_reshape=True,
                 window_size=[1024, 768],
                 cell_size=None,
                 show=True,
                 filename=None,
                 label=None,
                 show_ids=True,
                 mesh_color="grey",
                 ambient=0.8,
                 diffuse=0.5,
                 edge_color=None,
                 close_plotter=True
                ):
    """Plots a dataframe of buildings. If `cell_size` is set, then the `window_size` is ignored."""
    
    size_x = shape[0]
    size_y = shape[1]

    show_n = size_x * size_y
    
    if allow_reshape:
        if len(sample) == 1:
            size_x = 1
            size_y = 1
            show_n = 1
        elif len(sample) < show_n:
            size_x, size_y = get_shape(len(sample))
            show_n = size_x * size_y
    
    if len(sample) > show_n:
        sample = sample.sample(n = show_n)
    
    # If cell_size is set, resize the window respectively
    if not cell_size is None:
        window_size = [size_y*cell_size[0], size_x*cell_size[1]]
        
    p = pv.Plotter(window_size=window_size, shape=(size_x, size_y))
    
    p.background_color = "white"
    
    if not label is None:
        p.add_text(label, color="black")

    selected_ids = sample["id"]

    i = 0
    j = 0
    for objid in selected_ids:
        p.subplot(i, j)

        mesh, geom, verts = load_building(objid, tile_csv=df)
        p.add_mesh(mesh, color=mesh_color, ambient=ambient, diffuse=diffuse)
        if show_ids:
            p.add_text(objid, font_size=10, position="lower_right", color="black")
        #TODO: Add an option to print sublabels here
        if not edge_color is None:
            p.add_mesh(mesh.extract_feature_edges(feature_angle=20), color=edge_color, line_width=2)

        j += 1
        if j > size_y - 1:
            i += 1
            j = 0
    
    p.reset_camera()
        
    if show:
        p.show()
    
    if not filename is None:
        p.save_graphic(filename)
    
    if close_plotter:
        p.close()
    
    return p

p = plot_cluster(19, window_size=[2048, 2048], **colour_palette["pastel_blue"], show_class=False, show_ids=False, close_plotter=False, edge_color="white")
# plot_cluster(1, shape=(2,3))
# plot_cluster(2)
# plot_cluster(2, shape=(5, 5), allow_reshape=False)

In [ ]:
p.save_graphic(rpath("~/figures/clusters/sample_200k_30n_average/19_blue.pdf"))

In [ ]:
out_folder = rpath("~/figures/clusters/sample_200k_30n_average")

import os

if not os.path.exists(out_folder):
    os.mkdir(out_folder)

labels = np.sort(pd.unique(clustering["cluster"]))
# labels = [4, 6, 3]

# colors = ["#54086b", "#ff0bac", "#00bec5", "#050833"]
# colors = ["#4f0000", "#d9ced6", "#303437", "#303437"]
# colors = ["#b6e2d3", "#fbd2c9","#fff4bd", "#81abbc"]
for i, c in enumerate(labels[6:]):
    print(f"Plotting {c}...")
#     color = colors[i]
    plot_cluster(c, cell_size=[683, 683], filename=f"{out_folder}/{c}.pdf", show_class=False, show_ids=False, edge_color=None)

print("Done!")